In [1]:
%run -i ./startup.py
%run -i ./functions/dc_transform.py
%run -i ./functions/dc_join.py
%run -i ./functions/dc_list.py
%run -i ./functions/dc_keys.py
#%run -i ./functions/datagov_spider.py

In [2]:
import re
import nltk
import shapefile
import numpy as np
import seaborn as sns
import geopandas as gpd
import ipyleaflet as ipl

from datetime import date
from unidecode import unidecode
from matplotlib import pyplot as plt

In [3]:
in_dir = '/home/josegois/data/maps/brazil/estados'
in_shp = in_dir + '/UFEBRASIL.shp'

br_estados = gpd.read_file(in_shp)

In [4]:
getl  = lambda df,i,v: df[df[i].isin(v)].reset_index(drop=True)
getin = lambda df,i,v: df[df[i].isin(np.unique(v))].reset_index(drop=True)
getre = lambda df,i,v: df[df[i].isin(list(filter(lambda i : re.findall(v,i),df[i])))].reset_index(drop=True)

In [5]:
def dc_data(key_pattern='^.*$'):
    data = []
    for key in dc_join.dict.keys():
        if re.search(key_pattern,key):
            data.append(dc_join.dict[key]['data'])
    if ( len(data) == 1 ):
        return data[0]
    else:
        return data

In [ ]:
def dc_notin(data, fields, exvals):
    n_ex       = np.empty((len(exvals),len(fields)))
    n_ex[:]    = 0
    
    id_include    = np.empty((dc_perspective.shape[0],),dtype=bool)
    id_include[:] = True
                    
    for k,key in enumerate(fields):
        for e,exval in enumerate(exvals):
            id_exval   = dc_perspective[key].isin([exval]).values 
            n_ex[e,k]  = sum(id_exval) 
            id_include = id_include & ( ~id_exval )

        data = dc_perspective.iloc[id_include]
    return (data,n_ex)

In [6]:
def levdist (A,B):
    n_a = len(A)
    n_b = len(B)

    states_distance = np.empty((n_a,n_b))

    for a in range ( n_a ):
        for b in range ( n_b ):
            states_distance[a,b] = nltk.edit_distance(A[a], B[b])
    return states_distance

In [7]:
dc_alias_re = {
    'discentes'                : '^dados_complementares_.*discentes$',
    'discentes_complementar'   : '.*socio_economicos_.*discentes*'
}

for dc_alias,dc_re in dc_alias_re.items():
    dc_join(db=db,dc=dc_re,alias=dc_alias)

/home/josegois/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [80]:
discentes               = dc_data('^discentes$').merge(dc_data('^discentes_'),on='id_discente')
discentes.estado_origem = discentes.estado_origem.astype('category')

In [81]:
id_exclude = discentes.estado.isin([np.nan,'Ignorado/exterior','Não Informado'])

In [192]:
(discentes,n_excluded) =  dc_notin(discentes, ['estado','estado_origem'], [np.nan, 'Ignorado/exterior','Não Informado'])

In [215]:
estado_decoder = lambda x: re.sub('[^a-z]','',unidecode(x.lower()))

br_estados['estado_decoded'] = br_estados.NM_ESTADO.apply(estado_decoder)
discentes['estado_decoded'] = discentes.estado.apply(estado_decoder)
discentes['estado_origem_decoded'] = discentes.estado_origem.apply(estado_decoder)

discentes = discentes.merge(br_estados[['CD_GEOCODU','estado_decoded']], left_on='estado_decoded', right_on='estado_decoded', suffixes=('','_br'))
discentes = discentes.merge(br_estados[['CD_GEOCODU','estado_decoded']], left_on='estado_origem_decoded', right_on='estado_decoded', suffixes=('','_br'))

In [230]:
br_geojson = {
        'type' : 'FeatureCollection',
        'features' : []
    }

for estado in br_estados.iloc:
    if 'Mul' in str(estado.geometry.__class__):
        coord = [list(map(lambda p: list(p), eg.exterior.coords)) for eg in estado.geometry]
    else:
        coord = [list(map(lambda p: list(p), estado.geometry.exterior.coords))]

    d = {
        'type' : 'Feature',
        'id' : estado.CD_GEOCODU,
        'properties' : {'name' : estado.NM_ESTADO},
        'geometry' : {
            'type' : 'Polygon',
            'coordinates' : coord
        }
    }
    br_geojson['features'].append(d)

In [ ]:
discentes.

In [291]:
discentes.id_discente.unique().shape

(40912,)

In [290]:
discentes.id_discente.unique().shape

(40912,)

In [240]:
discentes_states = discentes.groupby(['CD_GEOCODU']).count()['id_discente']

In [259]:
discentes_states_origin = discentes.groupby(['CD_GEOCODU_br']).count()['id_discente']

In [271]:
layer_1_dict = discentes_states.to_dict()

In [270]:
layer_2_dict = discentes_states_origin.to_dict()

In [ ]:
layer_2_dict

In [278]:
list(map(np.log10, layer_1_dict.values()))

[0.7781512503836436,
 0.6020599913279624,
 0.9542425094393249,
 0.3010299956639812,
 2.1172712956557644,
 0.7781512503836436,
 1.2304489213782739,
 1.8573324964312685,
 2.037426497940624,
 3.1559430179718366,
 5.3067573233287435,
 3.2605483726369795,
 2.708420900134713,
 1.919078092376074,
 1.4913616938342726,
 2.1172712956557644,
 2.075546961392531,
 1.7634279935629373,
 2.0086001717619175,
 2.5888317255942073,
 1.1139433523068367,
 1.462397997898956,
 2.0086001717619175,
 1.0,
 1.2304489213782739,
 0.9542425094393249,
 1.792391689498254]

In [243]:
from branca.colormap import linear

In [254]:
br_layer = ipl.Choropleth(
    geo_data=br_geojson,
    choro_data=layer_1_dict,
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'})

In [269]:
br_geojson

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': '11',
   'properties': {'name': 'RONDÔNIA'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-63.32720817710297, -7.9767202985609025],
      [-63.11838271119069, -7.9771071712613555],
      [-62.86802113185014, -7.975875225088725],
      [-62.86661657169343, -7.9758682974353405],
      [-62.860170104651736, -7.982323288487947],
      [-62.8533605664975, -7.987563267848408],
      [-62.84505574263902, -7.9865312516612335],
      [-62.84160978602776, -7.994972725582617],
      [-62.83283330582775, -7.994371601956008],
      [-62.825722219924785, -8.007521357980519],
      [-62.821003079448786, -8.014495549554901],
      [-62.81433557046368, -8.018363442788157],
      [-62.80999653329572, -8.02533963951183],
      [-62.801755578884666, -8.025656025200945],
      [-62.794301859196, -8.02754786405665],
      [-62.786200314580924, -8.026343001074288],
      [-62.77983729237996, -8.031259432848781],
      [-62.7

In [260]:
br_layer2 = ipl.Choropleth(
    geo_data=br_geojson,
    choro_data=discentes_states_origin.to_dict(),
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'})

In [268]:
m = ipl.Map(center = (-20,-50), zoom = 3)
#m.add_layer(br_layer)
m.add_layer(br_layer2)
m.add_control(ipl.FullScreenControl())
control = ipl.LayersControl(position='topright')
m.add_control(control)
m

Map(center=[-20, -50], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…